In [4]:
# Import Aer, transpile and execute
from qiskit import transpile
from qiskit_aer import AerSimulator  # as of 25Mar2025
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT, UnitaryGate, PhaseGate, RZGate
from qiskit.quantum_info import Operator

import sys
sys.path.append('../') # Relative imports in jupyter notebooks are also a mess, but that´s a whole other story
from Hamiltonians_and_Lindbladians.utils_hamiltonian_simulation import *
from qiskit.visualization import plot_histogram, plot_state_city
from sympy import Matrix, latex
from IPython.display import display, Math
import matplotlib.pyplot as plt

## Preface: quantum vs non-quantum Ising models:

If one were to Google "Ising Model", like I was for the past couple hours, there is a high chance one will encounter two different formulations of the Ising model: The "classical" Ising model, which defines a Hamiltonian as

$$ H = -j \sum_{<i,j>} s_i s_j - h \sum_i s_i $$

where $s_i$ are the spins, $j$ is the coupling constant, and $h$ is the external magnetic field. The sum is over nearest neighbors.
The second formulation is the quantum Ising model, which is defined as

$$ H = -j \sum_{<i,j>} \sigma_i^z \sigma_j^z - h \sum_i \sigma_i^x $$

where $\sigma_i^z$ and $\sigma_i^x$ are the Pauli matrices. The sum is over nearest neighbors. 

The quantum Ising model is a quantum mechanical version of the classical Ising model, and it describes the behavior of spins in a magnetic field. The classical Ising model is a statistical mechanics model that describes the behavior of spins in a lattice, and it is used to study phase transitions and critical phenomena.

The quantum Ising model is a generalization of the classical Ising model, and it includes quantum effects such as tunneling and entanglement. The quantum Ising model can be solved using various methods, including mean-field theory, renormalization group theory, and numerical simulations.

(Obviously) I will focus on the quantum Ising model in this notebook, as it maps amazingly naturally to the quantum circuit model of computation. 


On a more superficial note, a  nicer notation (in my optinion at least) is to write the Hamiltonian as
$$
H_\mathrm{Ising} = - J \sum_{j=1}^{N-1} Z^{(j)} Z^{(j+1)} + g\sum_{j=1}^N X^{(j)}
$$

such that $\forall j \in \{1, \ldots, N\} $ we have $Z^{(j)}$ and $X^{(j)}$ are the Pauli matrices acting on the $j$-th qubit - aka we actually have an n-fold tensor product $H_j = \bigotimes_{j=0}^N \sigma_j\,, \quad \sigma_j \in \{I, X, Y, Z\}$ where every entry is just the identity matrix except for the $j$-th entry which is the specified Pauli matrix.

Check this souce out: https://people.maths.bris.ac.uk/~csxam/teaching/qc2020/lecturenotes.pdf


In [33]:
def generate_ising_hamiltonian(num_qubits: int, J, g) -> Union[SparsePauliOp, Pauli]:
    z_terms = []
    z_coeffs = []
    
    # ZZ interaction terms
    for j in range(num_qubits):
        pauli_string = ['I'] * num_qubits
        pauli_string[j] = 'Z'
        pauli_string[(j + 1) % num_qubits] = 'Z'  # Periodic boundary conditions
        z_terms.append("".join(pauli_string))
        z_coeffs.append(-J)  # Coefficient for ZZ interaction

    x_terms = []
    x_coeffs = []
    
    # X field terms
    for j in range(num_qubits):
        pauli_string = ['I'] * num_qubits
        pauli_string[j] = 'X'
        x_terms.append("".join(pauli_string))
        x_coeffs.append(-g)  # Coefficient for X term

    # Combine the Z and X terms into a single Hamiltonian
    all_terms = z_terms + x_terms
    all_coeffs = z_coeffs + x_coeffs

    return SparsePauliOp(all_terms, coeffs=all_coeffs)

def exponentiate_hamiltonian(hamiltonian: SparsePauliOp, time: float) -> Operator:
    """Exponentiates the Hamiltonian to obtain U = e^(-i H t)."""
    matrix = hamiltonian.to_matrix()
    unitary_matrix = scipy.linalg.expm(-1j * time * matrix)
    display(Math(latex(Matrix(unitary_matrix))))  # Display the unitary matrix in LaTeX format
    return Operator(unitary_matrix)

# example for 2 qubits
H = generate_ising_hamiltonian(2, 1.2, 1)
H

SparsePauliOp(['ZZ', 'ZZ', 'XI', 'IX'],
              coeffs=[-1.2+0.j, -1.2+0.j, -1. +0.j, -1. +0.j])

### (a) $g = 0$ (Classical Ising Limit)

The Hamiltonian reduces to a classical Ising model with only $Z_j Z_{j+1}$ interactions.

The ground states are ferromagnetic states:

$|00\ldots 0\rangle$ and $|11\ldots 1\rangle$  
(or in the Pauli-$X$ basis, $|\uparrow\uparrow\cdots\uparrow\rangle$ and $|\downarrow\downarrow\cdots\downarrow\rangle$).

The energy eigenvalues are:

$E = -JN,\ -J(N-4),\ \ldots,\ JN$

with different spin flip excitations.

### (b) $J = 0$ (Quantum Paramagnetic Limit)

The Hamiltonian reduces to a transverse field only:

$H = -g \sum_j X_j$

The ground state is a fully polarized state in the $X$-basis:

$|+\rangle^{\otimes N} = \frac{1}{\sqrt{2^N}} \sum_z |z\rangle$

Eigenvalues are:

$E = -gN,\ -g(N-2),\ \ldots,\ gN$


In [3]:
H = generate_ising_hamiltonian(2, 1.2, 1)

# Convert to matrix form
H_matrix = H.to_matrix()

# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eigh(H_matrix)

# Display eigenvalues and eigenvectors
latex_vecs = latex(Matrix(eigenvectors))  # Convert matrix to LaTeX format
display(Math(r"\text{Eigenvectors: } " + latex_vecs))
print("Eigenvalues: ", eigenvalues)

<IPython.core.display.Math object>

Eigenvalues:  [-3.12409987 -2.4         2.4         3.12409987]


## **Effect of MSB Choice in Quantum Phase Estimation (QPE)**

### **1. Overview**
Quantum Phase Estimation (QPE) extracts the phase $\theta $ in the eigenvalue equation:

$$
U | \psi \rangle = e^{2\pi i \theta} | \psi \rangle
$$

where $ U $ is a unitary matrix with eigenvalue $ e^{2\pi i \theta} $. The algorithm encodes $ \theta $ using a register of $ t $ ancilla qubits and controlled-$ U^{2^k} $ operations.

---

### **2. Controlled-$ U^{2^k} $ Order**
The choice of whether the **most significant bit (MSB) or least significant bit (LSB) is assigned to qubit 0** determines how powers of $ U $ are applied.

- **MSB-first (Standard Convention)**
  - Qubit $ 0 $ controls $ U^{2^0} $.
  - Qubit $ 1 $ controls $ U^{2^1} $.
  - Qubit $ t-1 $ controls $ U^{2^{t-1}} $.
  - The measurement outcome is in **standard binary order**, directly yielding:

    $$
    \theta \approx \frac{j}{2^t}
    $$

    where $j $ is the measurement outcome in binary.

- **LSB-first (Alternative Convention)**
  - Qubit $ 0 $ controls $ U^{2^{t-1}} $.
  - Qubit $ 1 $ controls $ U^{2^{t-2}} $.
  - Qubit $ t-1 $ controls $ U^{2^0} $.
  - The measurement outcome is in **bit-reversed order**, yielding:

    $$
    \theta \approx \frac{j_{\mathrm{rev}}}{2^t}
    $$

    where $ j_{\mathrm{rev}} $ is the bit-reversed measurement outcome.
    - This requires additional processing to recover the original phase estimate.

## **5. Conclusion**
- The **MSB-first convention** directly encodes \( \theta \) in standard binary fraction form.
- The **LSB-first convention** leads to a **bit-reversed result**, requiring additional processing to recover \( \theta \).


In [2]:


def test_many_implementations_of_qpe(U, num_qubits, num_ancilla, eigenstate):
    qpe_circ_1 = standard_qpe(unitary=U, eigenstate=eigenstate, num_ancilla=num_ancilla, swap=True)
    qpe_circ_2 = standard_qpe(unitary=U, eigenstate=eigenstate, num_ancilla=num_ancilla, swap=False)
    qpe_circ_3 = standard_qpe_v2(unitary=U, eigenstate=eigenstate, num_ancilla=num_ancilla, num_target=num_qubits, swap=True)
    qpe_circ_4 = standard_qpe_v2(unitary=U, eigenstate=eigenstate, num_ancilla=num_ancilla, num_target=num_qubits, swap=False)

    return [qpe_circ_1, qpe_circ_2, qpe_circ_3, qpe_circ_4]

    